In [29]:
import pandas as pd
import numpy as np
import os

In [30]:
# Sample data

DEFAULT_DATE_RANGE = ('2023-01-01', '2025-01-01')

def generate_timeseries_df():
    start, end = DEFAULT_DATE_RANGE
    idx = pd.date_range(start=start, end=end, freq="MS") # freq month start
    n_points = len(idx)
    rng = np.random.default_rng(np.random.randint(1, 99))
    base = rng.normal(loc=5500, scale=1500, size=n_points)
    trend = np.linspace(0, 2000, n_points)
    values = np.maximum(base + trend, 1000).astype(float)
    values = np.minimum(values, 10000).round(2)
    df = pd.DataFrame({"value": values})
    df.insert(0, "date", idx)
    return df
    
def generate_multiline_df():
    start, end = DEFAULT_DATE_RANGE
    idx = pd.date_range(start=start, end=end, freq="MS")

    rng = np.random.default_rng(np.random.randint(1, 99))
    region_names = ["North", "South", "East", "West"]
    frames: list[pd.DataFrame] = []
    for i, name in enumerate(region_names):
        base = 700_000 + i * 120_000
        # Alternate trend direction for some lines
        if i % 2 == 0:
            trend_slope = 400_000 + i * 50_000
            trend = np.linspace(0, trend_slope, len(idx))
        else:
            trend_slope = 400_000 + i * 50_000
            # Down then up: negative slope for first half, positive for second half
            half = len(idx) // 2
            trend = np.concatenate([
                np.linspace(0, -trend_slope, half),
                np.linspace(-trend_slope, trend_slope, len(idx) - half)
            ])
        noise = rng.normal(loc=0, scale=60_000, size=len(idx))
        values = np.maximum(base + trend + noise, 0).round(2)
        frames.append(pd.DataFrame({"date": idx, "region": name, "value": values}))

    df = pd.concat(frames, ignore_index=True)
    return df

In [31]:
bar = generate_timeseries_df()
single_line = generate_timeseries_df()
multi_line = generate_multiline_df()

# Bar chart

In [32]:
bar.head()

,date,value
0,2023-01-01,6863.72
1,2023-02-01,6710.43
2,2023-03-01,5628.36
3,2023-04-01,6728.76
4,2023-05-01,4856.90


# Single line chart

In [33]:
single_line.head()

,date,value
0,2023-01-01,3786.86
1,2023-02-01,3979.33
2,2023-03-01,4530.78
3,2023-04-01,6902.99
4,2023-05-01,4276.43


# Multi line chart

In [34]:
multi_line.head()

,date,region,value
0,2023-01-01,North,676281.92
1,2023-02-01,North,732501.56
2,2023-03-01,North,769761.03
3,2023-04-01,North,691670.41
4,2023-05-01,North,812726.52


In [35]:
# Export the final dfs needed for the charts
os.makedirs('output', exist_ok=True)
bar.to_csv('output/bar.csv', index=False)
single_line.to_csv('output/single_line.csv', index=False)
multi_line.to_csv('output/multi_line.csv', index=False)
